In [2]:
#using Gadfly, ColorTypes

In [3]:
using StatsBase

In [23]:
#Aquí se pueden hacer cambios para realizar varios muestreos
side = 50
size = side^2
init_nmols = 300
@assert init_nmols <= size "init_nmols > size" #Comprobar que no haya más partículas que sitios
ntrials = convert(Int, floor(side^4))# Es buena idea que vaya como n^3
#ntrials = 120
nsamples = size
#nsamples = 1
j_ising = -1.0
μ = 0.0
t_env = 0.7
zz = exp(μ/t_env)/sqrt(t_env^3)
iters = 6
temperatures = linspace(0.5, 5, 10)
const direc = pwd()*"/ising_gc2/"

"/home/rafael/Documentos/Tesis/Programas/Servicio social/ising_gc2/"

In [5]:
function change_spin()
    return rand(Int32[-1, 1])
end

change_spin (generic function with 1 method)

In [6]:
function initialize(side, init_nmols)

    spins = Vector{Int32}(side*side)
    neighbors = Vector{Vector{Int64}}(side*side)
    occupancy = Vector{Bool}(side*side)

    # Seleccionar partículas iniciales: init_mols es una lista
    init_mols = sample(1:(side*side), init_nmols, replace = false)
    for jj in 1:side
        for ii in 1:side
            index = (jj-1)*side + ii
            spins[index] = change_spin()
            # Definir los índices de los vecinos
            upper = mod(jj, side)*side + ii
            lower = mod(jj - 2, side)*side + ii
            left  = (jj-1)*side + mod(ii - 2, side) + 1
            right = (jj-1)*side + mod(ii, side) + 1 
            neighbors[index] = Int64[upper, lower, left, right]
            
            #if index in init_mols
            #    occupancy[index] = true
            #else
            #    occupancy[index] = false
            #end
            #Lo anterior se puede resumir en:
            occupancy[index] = index in init_mols #Se usa el valor de verdad de la evaluación
        end
    end
    
    return spins, neighbors, occupancy
end

initialize (generic function with 1 method)

spins, neighbors, occupancy = initialize(side, init_nmols)

In [7]:
function add_spin(occupancy)
    free_sites = find(!occupancy)
    if length(free_sites) == 0 # Si todos los sitios están ocupados
        return -1
    end
    trial_idx = rand(free_sites)
    return trial_idx
end

add_spin (generic function with 1 method)

In [8]:
function delete_spin(occupancy)
    occupied_sites = find(occupancy)
    if length(occupied_sites) == 0 # Si todos los sitios están vacíos
        return -1
    end
    trial_idx = rand(occupied_sites)
    return trial_idx
end

delete_spin (generic function with 1 method)

In [9]:
function move_spin(occupancy, size)
    occupied_sites = find(occupancy)
    # Si todos los sitios están  ocupados o  vacíos, no se puede realizar nada
    if length(occupied_sites) in [size, 0] 
        return -1, -1
    end
    old_idx = rand(occupied_sites)
    free_sites = find(!occupancy)
    trial_idx = rand(free_sites)
    return old_idx, trial_idx
end

move_spin (generic function with 1 method)

In [36]:
function montecarlo_cycle(spins, neighbors, occupancy, size, ntrials, nsamples, j_ising, t_env, zz)
    """
    Repeats many attepts to change the system and sample some quantities
    """
    energies = Float64[]
    magnet = Float64[] #Aquí porque estoy dividiendo
    te = 0.0
    for ii in 1:size
        neighbors_spins = spins[neighbors[ii]].*occupancy[neighbors[ii]]
        te = te + potential_energy(occupancy[ii]*spins[ii], neighbors_spins, j_ising)
    end
    te = te/2.0
    push!(energies, te)
    m1 = magnetization(spins)/size
    push!(magnet, m1)
    for ii in 1:ntrials
        energy_change = montecarlo!(spins, neighbors, occupancy, size, j_ising, t_env, zz)
        te += energy_change
        #if ii >= trials_before_sample
            #counter = 1
        if ii%nsamples == 0
                # Samples total magnetization
            m1 = magnetization(spins)/size
            push!(magnet, m1)
                # Samples total energy
            push!(energies, te)
            #    counter += 1
        end
    end
    return energies, magnet
end

montecarlo_cycle (generic function with 1 method)

In [32]:
function montecarlo!(spins, neighbors, occupancy, size, j_ising, t_env, zz)
    """
    Tries to perform a change in the system, like changing a spin direction
    """
    r = rand()
    nmol = sum(occupancy)
    
    # añadir espín
    if 0 <= r < 0.2
        idx = add_spin(occupancy)
        # Por si no hay espacios libres para añadir otro espín
        if idx == -1
            return 0.0
        end
        old_energy = 0.0
        trial_spin = change_spin()
        neighbors_spins = spins[neighbors[idx]].*occupancy[neighbors[idx]]
        trial_energy = potential_energy(trial_spin, neighbors_spins, j_ising)
        energy_diff = trial_energy - old_energy
        energy_change = 0.0
        if metropolis_step(energy_diff/t_env - log(zz*size/(nmol + 1)))
            occupancy[idx] = true
            spins[idx] = trial_spin
            energy_change = energy_diff
#        else
#            occupancy[idx] = false
        end
        #println("¿Se añadió? ", metropolis_step(energy_diff/t_env - log(zz*size/(nmol + 1))),
        #    "   Cambio por añadir: ", energy_change)
        
    # remover espín
    elseif 0.2 <= r < 0.4
        idx = delete_spin(occupancy)
        # Por si no quedan espines en el sistema
        if idx == -1
            return 0.0
        end
        neighbors_spins = spins[neighbors[idx]].*occupancy[neighbors[idx]]
        old_energy = potential_energy(spins[idx], neighbors_spins, j_ising)
        trial_energy = 0.0
        energy_diff = trial_energy - old_energy
        energy_change = 0.0
        if metropolis_step(energy_diff/t_env - log(nmol/(zz*size)))
            occupancy[idx] = false
            energy_change = energy_diff
        end
        #println("¿Se removió? ", metropolis_step(energy_diff/t_env - log(zz*size/(nmol + 1))),
        #    "   Cambio por remover: ", energy_change)        
    
    # mover espín a posición vacía aleatoria
    elseif 0.4 <= r < 0.7
        old_idx, trial_idx = move_spin(occupancy, size)
        # Por si no hay espacios para mover un espín
        if old_idx == -1
            return 0.0
        end
        old_neighbors_spins = spins[neighbors[old_idx]].*occupancy[neighbors[old_idx]]
        old_energy = potential_energy(spins[old_idx], old_neighbors_spins, j_ising)
        
        #Estos dos se cambian por si se mueve a un sitio vecino
        occupancy[old_idx] = false
        occupancy[trial_idx] = true
        
        trial_neighbors_spins = spins[neighbors[trial_idx]].*occupancy[neighbors[trial_idx]]
        trial_energy = potential_energy(spins[old_idx], trial_neighbors_spins, j_ising)
        
        energy_diff = trial_energy - old_energy
        energy_change = 0.0
        if metropolis_step(energy_diff/t_env)
            #occupancy[old_idx] = false
            spins[trial_idx] = spins[old_idx]
            #occupancy[trial_idx] = true
            energy_change = energy_diff
        else
            # Si se intentó mover a un sitio vecino y no se aceptó el movimiento,
            # se regresan a sus valores originales
            occupancy[old_idx] = true
            occupancy[trial_idx] = false
        end
        #println("¿Se movió? ", metropolis_step(energy_diff/t_env - log(zz*size/(nmol + 1))),
        #    "   Cambio por mover: ", energy_change)
    
    # Solamente cambiar el espín (creo que sí es necesario)
    else
        idx = delete_spin(occupancy) #Porque encuentra los sitios ocupados
        if idx == -1
            return 0.0
        end
        neighbors_spins = spins[neighbors[idx]].*occupancy[neighbors[idx]]
    
        old_energy = potential_energy(spins[idx], neighbors_spins, j_ising)
        trial_spin = -spins[idx]
        trial_energy = potential_energy(trial_spin, neighbors_spins, j_ising)

        energy_diff = trial_energy - old_energy
        energy_change = 0.0
        if metropolis_step(energy_diff/t_env)
            spins[idx] = trial_spin
            energy_change = energy_diff
        end
        #println("¿Se volteó? ", metropolis_step(energy_diff/t_env - log(zz*size/(nmol + 1))),
        #    "   Cambio por voltear: ", energy_change)
    end
    return energy_change
end

montecarlo! (generic function with 1 method)

In [12]:
# usé @code_warntype después de poner neighbors_spins{Vector{Int64}} y spins{Float64} y no arrojó problemas
function potential_energy(central_spin, neighbors_spins, j_ising::Float64)
    #convol_mat = [0, 1, 0, 1, 0, 1, 0, 1, 0]
    #convol_mat describe cuáles vecinos sí interactúan con el espín de interés.
    #La multiplicación entrada por entrada elige cuáles espines sí interactúan.
    #sum_energies = 0
    #for ii in 1:9
        #sum_energies = sum_energies + neighbors_spins[ii]*convol_mat[ii]
    #end
    #return j_ising*central_spin*sum(neighbors_spins.*convol_mat)
    return j_ising*central_spin*sum(neighbors_spins)
end

potential_energy (generic function with 1 method)

In [13]:
function magnetization(spins)
    return sum(spins)
end

magnetization (generic function with 1 method)

In [14]:
function metropolis_step(exp_arg)
    """
    Returns True if the trial configuration goes towards a region with higher probability
    or it is given the chance to explore regions with fewer probability
    """
    if exp_arg < 0.0
        return true # Se actualiza el estado del sistema
    else
        w = exp(-exp_arg)
        if rand() < w # Energía: w
            return true # También se actualiza el estado del sistema
        else
            return false
        end
    end
end

metropolis_step (generic function with 1 method)

### Lo que sigue ya podría ser el programa principal

function init_loop(temperatures)
    energies_per_temp = Vector()
    magnets_per_temp = Vector()
    for temp in temperatures
        ens_energies = Vector{Float64}()
        ens_magnets = Vector{Float64}()
        for ii in 1:iters
            spins, neighbors, occupancy = initialize(side, init_nmols)
            spins, occupancy, energies, magnet = montecarlo_cycle(spins, neighbors, occupancy, size, ntrials, nsamples, t_env, zz)
            ens_energy = sum(energies)/length(energies)
            ens_magnet = sum(magnet)/length(energies)
            push!(ens_energies, ens_energy)
            push!(ens_magnets, ens_magnet)
        end
    push!(energies_per_temp, ens_energies)
    push!(magnets_per_temp, ens_magnets)
    end
    return energies_per_temp, magnets_per_temp
end

In [37]:
spins, neighbors, occupancy = initialize(side, init_nmols)
spins

2500-element Array{Int32,1}:
 -1
 -1
  1
  1
 -1
  1
 -1
  1
 -1
  1
  1
 -1
 -1
  ⋮
  1
 -1
 -1
 -1
 -1
  1
 -1
 -1
  1
  1
  1
  1

In [38]:
occupancy

2500-element Array{Bool,1}:
  true
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
     ⋮
 false
 false
  true
  true
  true
 false
  true
  true
 false
 false
 false
 false

In [39]:
#spins, occupancy, energies, magnet = montecarlo_cycle((spins, neighbors, occupancy, size, ntrials, nsamples, t_env, zz)
@time energies, magnet = montecarlo_cycle(spins, neighbors, occupancy, size, ntrials, nsamples, j_ising, t_env, zz)
#@code_warntype montecarlo_cycle(spins, neighbors, occupancy, size, ntrials, nsamples, j_ising, t_env, zz)
# Me parece que lo rojo significa que la función montecarlo no devuelve variables del mismo tipo.
# En este caso devuelve ::Union{Array{Bool,1},Array{Int32,1},Float64} y no sé si debería cambiar eso

123.816129 seconds (35.75 M allocations: 98.203 GB, 7.09% gc time)


([-7.0,-305.0,-720.0,-1239.0,-1787.0,-2371.0,-3087.0,-3452.0,-3528.0,-3660.0  …  -4764.0,-4768.0,-4768.0,-4764.0,-4764.0,-4768.0,-4764.0,-4760.0,-4764.0,-4764.0],[0.0504,0.0328,0.012,-0.0104,-0.0368,-0.048,-0.0456,-0.0568,-0.0584,-0.0568  …  -0.0056,-0.0104,-0.0104,-0.0096,-0.0088,-0.0104,-0.012,-0.0112,-0.0136,-0.0128])

In [40]:
#7.876384 seconds (7.42 M allocations: 1.642 GB, 4.56% gc time) 20x20 anterior y no siempre llegaba al mínimo de energía
#0.702974 seconds (1.05 M allocations: 465.859 MB, 14.16% gc time) 20x20 nuevo y esta vez llegó al mínimo de energía
#2.116618 seconds (2.51 M allocations: 1.113 GB, 11.64% gc time) 20x20 con side^4.3
#114.139761 seconds (47.22 M allocations: 98.459 GB, 6.78% gc time) 50x50 y casi alcanza el mínimo con ntrials = side^4
#401.833861 seconds (151.47 M allocations: 318.393 GB, 7.07% gc time) 50x50 con ntrials^4.3 y le falta

In [41]:
spins

2500-element Array{Int32,1}:
 -1
 -1
 -1
 -1
 -1
 -1
 -1
 -1
 -1
 -1
 -1
 -1
 -1
  ⋮
  1
  1
  1
 -1
 -1
 -1
 -1
 -1
 -1
 -1
 -1
 -1

In [42]:
occupancy

2500-element Array{Bool,1}:
 true
 true
 true
 true
 true
 true
 true
 true
 true
 true
 true
 true
 true
    ⋮
 true
 true
 true
 true
 true
 true
 true
 true
 true
 true
 true
 true

In [43]:
magnet

2501-element Array{Float64,1}:
  0.0504
  0.0328
  0.012 
 -0.0104
 -0.0368
 -0.048 
 -0.0456
 -0.0568
 -0.0584
 -0.0568
 -0.0624
 -0.0664
 -0.08  
  ⋮     
 -0.0032
 -0.0048
 -0.0056
 -0.0104
 -0.0104
 -0.0096
 -0.0088
 -0.0104
 -0.012 
 -0.0112
 -0.0136
 -0.0128

In [44]:
energies

2501-element Array{Float64,1}:
    -7.0
  -305.0
  -720.0
 -1239.0
 -1787.0
 -2371.0
 -3087.0
 -3452.0
 -3528.0
 -3660.0
 -3732.0
 -3812.0
 -3858.0
     ⋮  
 -4768.0
 -4768.0
 -4764.0
 -4768.0
 -4768.0
 -4764.0
 -4764.0
 -4768.0
 -4764.0
 -4760.0
 -4764.0
 -4764.0

max_nmols = ncols*mrows
splist = Vector(max_nmols)
splist2 = Vector(max_nmols)
for ii in 1:max_nmols
    splist[ii] = particles[ii].spin*particles[ii].occupancy
    splist2[ii] = new_particles[ii].spin*new_particles[ii].occupancy
end

sys_img = spy(reshape(splist, ncols, mrows), Theme(background_color = "white"))
img = SVG(direc*"Configuración inicial.svg", 9inch, 6inch)
draw(img, sys_img)

sys_img2 = spy(reshape(splist2, ncols, mrows), Theme(background_color = "white"))
img2 = SVG(direc*"Configuración final.svg", 9inch, 6inch)
draw(img2, sys_img2)

descenso = plot(x = linspace(0, 240, length(energies)), y = energies, 
    Guide.title("Descenso de energía para Metropolis MC 100x100"),
    Theme(background_color = "white"))
draw(PNG(direc*"Descenso de energía para Metropolis MC 100x100", 8inch, 6inch), descenso)

@time avg_energies, avg_magnets = init_loop(temperatures)

x_axis = vcat([temp*ones(iters) for temp in temperatures]...)
y_energies = vcat(avg_energies...)
eg1 = plot(x = x_axis, y = y_energies,
    Guide.xlabel("Temperatura"), Guide.ylabel("Energía"),
    Guide.title("E vs T para el modelo de Ising de "*string(ncols)*"x"*string(ncols)*" con J = -1"),
    Theme(background_color = RGB(1,1,1)))

y_magnets = vcat(avg_magnets...)
mg1 = plot(x = x_axis, y = y_magnets,
    Guide.xlabel("Temperatura"), Guide.ylabel("Magnetización"),
    Guide.title("M vs T para el modelo de Ising de "*string(ncols)*"x"*string(ncols)*" con J = -1"),
    Theme(background_color = RGB(1,1,1)))

writedlm(direc*"ising_gc.txt", energies)

en = plot(x = 1:4000:length(energies), y = energies[1:4000:end], 
    Guide.title("Grand Canonical Montecarlo"), 
    Theme(background_color = "white"))
img3 = SVG(direc*"energías.svg", 9inch, 6inch)
draw(img3, en)